# Basic Statistics (Part 3)

**Prerequisite**

* $\texttt{numpy}$ and $\texttt{matplotlib}$.
* Basic Statistics (Part 1 and 2)

**New skills**

* Probability notation and basic rules
* Likelihood functions
* Fitting a model

In [1]:
# Let's start with importing our packages
import numpy as np
import scipy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# We can beautify our plots by changing the matplotlib settings a little
plt.rcParams['font.size'] = 18
matplotlib.rcParams['axes.linewidth'] = 2
matplotlib.rcParams['font.family'] = "serif"

## Part 1: probability functions


**Exercise 1**

The Gaussian Probability Density Function (PDF) is
$$p(x) = \frac{1}{\sqrt{2\pi} \sigma} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$

Write a function for the Gaussian pdf: `def gaussian_pdf(x, mu, sigma)` that returns `p(x)`. Then plot the Gaussian pdf for `-4 < x < 4` alongside a histogram of random draws from the Gaussian distribution (with `density=True`). Repeat this process for $\sigma = 0.5, 1.0$ (all with $\mu = 0$).

How many random draws do you need for the histogram to closely match `p(x)`? Does `p(x)` match the histogram if we instead set `density=False`?

*Bonus:* use for loops

In [3]:
def gaussian_pdf(x, mu, sigma):
    # your answer here

# Part 2: fitting models to data

## Linear fit

Let's revist the data on football players from earlier. We already saw that height and weight are correlated in the data (taller players tend to weight more). While we can clearly see the trend 'by eye,' is there a way to describe this correlation mathematically? For example, we may want to ask: "what is the typical weight of players who are 6 feet tall?"

In astronomy, we often find ourselves asking questions like that. For example, how does the size of a star depend on it's brightness? or how does the shape of a galaxy depend on its age?

In this section, we will learn how to find the mathemtical function that best describes a set of data.

Run the code below to load in the height and weights from the football player database.

In [4]:
# Let's load in the data
import os
from google.colab import drive
from astropy.table import Table

drive.mount('/content/drive/')
os.chdir('/content/drive/Shareddrives/AST207/data')

cat = Table.read('./players_age.csv')

height = cat['height_inches']
weight = cat['weight']

height = np.array(height)
weight = np.array(weight)

Mounted at /content/drive/


The simplist relation between two variables `x` and `y` is the linear function:
$$ y = m * x + b $$
in our case, `x` is a player's height and `y` is the player's weight.

**Exercise 2**

Make the function `def linear(x,m,b)` that returns `y=m*x+b`. For `65 < x < 80` plot `linear(x, m, b)` alongside the observed heights and weights. Choose `m,b` such that the line best matches the observed data.

Be sure to include labels.

In [ ]:
def linear(x,m,b):
  # your answer here

Finding values of `m,b` (the slope and intercept of our linear function) by hand wasn't too difficult but we have a few problems:


1.   How do we convince our friends that we found the **best** values of `m,b`?
2.   What if we have a function with lots of variables? Then finding the best by hand is no longer feasible.

Thankfully, we don't have the find the best values by hand, we can ask our computers to help us. To have the computer find the best fitting line to the data, we'll follow a two step process:


1.   Define a function that tells us how well our line matches the data. Let's call this our *objective function*.
2.   Find the values of `m,b` that optimize our objective function.

The simplist objective function would be to calculate the difference between our model's prediction for a player's weight given their height and the player's actual weight. If we calculate the difference between the model and the data for all the players, we'll know how closely we match the data. Mathematically this looks like:

$$ \mathrm{Simple~Objective~Function}(m,b)  = | \mathrm{linear}(x_1,m,b) - y_1 | + | \mathrm{linear}(x_2,m,b) - y_2 | + \dots $$

the "$\dots$" means that we sum the difference between the model and the data for all the players in the table. The "| |" mean we are taking the absolute value (for example $| -1 |=1$).

This simple function is close but let's make two small changes:
1. Let's consider the **fractional** difference between the model prediction and the observed data (this will make sure we can fit data will small and large numbers easily).
2. Let's take the square of the difference instead of absolute value. This is a common convention and will help us later on in the class.

Now our final objective function is:

$$ \mathrm{Objective~Function}(m,b)  =  \frac{[ \mathrm{linear}(x_1,m,b) - y_1 )]^2}{|y_1|} + \frac{[ \mathrm{linear}(x_2,m,b) - y_2 )]^2}{|y_2|}  + \dots $$

(this objective function is commonly known as $\chi^2$)

**Exercise 3**

Write a function that calculates the "Objective Function" for a given set of `x` and `y` data and the two parameters `m,b` of the linear function. Your function should have the form: `def objective_function(x,y,m,b)`

As a test, `objective_function(x=height,y=weight,m=10,b=-500)` should return `6216.87`

Hint: use `np.abs` to take the absolute value

In [ ]:
def objective_function(x,y,m,b):
    # your answer here

In [ ]:
# test your function:
objective_function(x=height,y=weight,m=10,b=-500)

**Exercise 4**

Use your `objective_function` to search for good values of `m,b`. Using `for` loops, calculate `objective_function` for $0<m<20$ and $-1000<b<0$. For each combination of `b,m` plot `b` vs. `m` on a scatter plot with the color set by `simple_objective_function(x,y,m,b)`. For an example of a scatter plot with the points colored by a third parameter, see the [`Getting started with arrays and plots in Python notebook`](https://jgreene100.github.io/Ast207/docs/intro_python/numpy_and_plotting.html). Try 50 values of `b` and 50 values of `m` (it should run in less than a second). For ease of reading, plot `np.log10` of the objective function.

In [ ]:
# your answer here

**Exercise 5**

From your grid search above, find the values of `m,b` that yield the lowest `simple_objective_function`. Plot the linear function using these parameters alongside the football data and your by-band linear function. Be sure to label your plot.

Hint: try using `np.argmin`

In [ ]:
# Your answer here

## Fitting astronomical data

Earlier, we looked at data from the Gaia space telescope. Gaia carefully observes the positions and properties of stars in our galaxy. The code below loads in the Gaia data for all stars within 15 parsec of the Sun.

In [5]:
# Let's load in the data
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/Shareddrives/AST207/data')

# path to the table
path = 'gaia_15pc.csv'

# loading the table; this function assumes a 'comma separated file (aka csv)'
gaia = pd.read_csv(path)

# simple errors
gaia.loc[:,'color']     = gaia.gmag - gaia.rmag
gaia.loc[:,'color_err'] = 0.04
gaia.loc[:,'teff_err']  = 10. # K

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Exercise 6**

In a previous activity, we searched for which properties of a star are correlated (like how football players heights and weights are correlated). Let's revisit that activity here. Below, plot `teff` versus `color` for the Gaia data (`teff` is the temperature of the star and `color` is the difference between the stars' `g` and `r` magnitudes). Be sure to label your plot.

In [ ]:
# Your plot here

Cool! We have a clear observational result: color and temperature of a star are strongly correlated.

However, we still don't know *how* color and temperature are related. Is the relation linear, quadratic, or perhaps much more complex? Answering this question will greatly increase our understanding of the stars in our catalog (it will also help our theorist friends try to *explain* what's happening here).

**Exercise 7**

Like we did for the football data, fit the `teff` and `color` data with a linear function. Use the "grid search" method we used for the football data. Try fitting the data by hand first to choose appropriate ranges of `m,b` to search.

Print out the best fit `m,b` and plot the best-fit line against the observed data.

Hint: the slope (`m`) will be small. Try `m = -1e-4` to start.

In [ ]:
# Your answer

**We just fit a model to astronomical data!** This is a critical tool in modern astronomy. But so far, we can only find the best solution by a time consuming "grid search." Let's find a better way.

Many algorithms have been developed to find the parameters that minimize a function (our grid search is one example). Let's use `scipy.optimize.minimize`.

**Exercise 8**

Run the code below to use `scipy.optimize.minimize`. Then plot the best fit linear function from `scipy.optimize.minimize` alongside the observed data and your "grid search" fit.

In [ ]:
def general_objective_function(theta, fn, x, y):

  y_model = fn(x,*theta)

  return np.sum(np.power(y-y_model,2) / np.abs(y) )

out = scipy.optimize.minimize(general_objective_function, [-1e-4, 2], bounds = [[-1e-3,0], [0,10]], args = (linear, gaia.teff, gaia.color) )
m_fit, b_fit = out.x

In [ ]:
# Your plot here

We can now perform a linear fit and even can do it quicky using `scipy`. However, the above plots show that a linear fit is not a great match to the data. So what is?

Fortunately, `general_objective_function` (which we provided for the activity above) allows us to change the function we are fitting...

**Exercise 9**

Let's try a model where color is *inversely* proportional to temperature:
$$ \mathrm{color} = m / T + b $$
Write a function `def inverse(x, m, b)` that returns the inverse function. Then use `scipy` to find the best fit parameters. Plot the best fit inverse function alongside the observed data and the best fit linear function.


In [ ]:
def inverse(x, m, b):
  # your code

# use scipy to find the best fit parameters

In [ ]:
# Your plot